In [1]:
import numpy as np
import pandas as pd
df_movies = pd.read_csv('movies.csv')
df_links = pd.read_csv('links.csv')
df_ratings = pd.read_csv('ratings.csv')
df_tags = pd.read_csv('tags.csv')

In [2]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df_tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [5]:
import math 
tf = df_tags.groupby(['movieId','tag'], as_index=False, sort=False)\
         .count()\
         .rename(columns = {'userId':'tag_count_tf'})[['movieId','tag','tag_count_tf']]
tag_distinct = df_tags[['tag','movieId']].drop_duplicates()


In [6]:
df = tag_distinct.groupby(['tag'], as_index=False, sort=False)\
                 .count()\
                 .rename(columns = {'movieId':'tag_count_df'})[['tag','tag_count_df']]


In [7]:
idf = math.log10(len(np.unique(df_tags['movieId'])))
df['idf'] = idf-np.log10(df['tag_count_df'])


In [8]:
tf = pd.merge(tf, df, on='tag', how='left', sort=False)
tf['tf-idf'] = tf['tag_count_tf']*tf['idf']


In [9]:
tf[['movieId','tag','tf-idf']].head()

,movieId,tag,tf-idf
0,60756,funny,5.622700
1,60756,Highly quotable,2.719331
2,60756,will ferrell,8.686268
3,89774,Boxing story,3.196453
4,89774,MMA,3.196453


In [10]:
vect_length = tf.loc[:,('movieId','tf-idf')]

In [11]:
vect_length['tf-idf-sq'] = vect_length['tf-idf']**2
vect_length = vect_length.groupby(['movieId'], as_index=False, sort=False)\
                   .sum()\
                   .rename(columns = {'tf-idf-sq':'tf-idf-sq-total'})[['movieId','tf-idf-sq-total']]
vect_length['vect_length'] = np.sqrt(vect_length[['tf-idf-sq-total']].sum(axis=1))

tf = pd.merge(tf, vect_length, on='movieId', how='left', sort=False)
tf['tag_vec'] = tf['tf-idf']/tf['vect_length']


In [12]:
tf[['movieId','tag','tf-idf','vect_length','tag_vec']].head()


,movieId,tag,tf-idf,vect_length,tag_vec
0,60756,funny,5.622700,10.887728,0.516425
1,60756,Highly quotable,2.719331,10.887728,0.249761
2,60756,will ferrell,8.686268,10.887728,0.797804
3,89774,Boxing story,3.196453,5.368248,0.595437
4,89774,MMA,3.196453,5.368248,0.595437


In [13]:
ratings_filter = df_ratings[df_ratings['rating']>=3]
user_distinct = np.unique(df_ratings['userId'])
user_tag_pref = pd.DataFrame()
i = 1

In [14]:
userId = 89

In [15]:
user_index = user_distinct.tolist().index(userId)

for user in user_distinct[user_index:user_index+1]:
    
    if i%30==0:
        print ("user: ", i , "out of: ", len(user_distinct))
            
    user_data= ratings_filter[ratings_filter['userId']==user]
    user_data = pd.merge(tf,user_data, on = 'movieId', how = 'inner', sort = False)
    user_data_itr = user_data.groupby(['tag'], as_index = False, sort = False)\
                             .sum()\
                             .rename(columns = {'tag_vec': 'tag_pref'})[['tag','tag_pref']]
    user_data_itr['user']=user
    user_tag_pref = user_tag_pref.append(user_data_itr, ignore_index=True)
    i=i+1


In [16]:
user_tag_pref.head()

,tag,tag_pref,user
0,animation,0.279565,89
1,Disney,1.235981,89
2,funny,0.811149,89
3,original,0.368530,89
4,Pixar,1.692234,89


In [17]:
import pandas as pd

user_distinct = np.unique(ratings_filter['userId'])
tag_merge_all = pd.DataFrame()
i = 1


In [18]:
userId = 89

In [19]:
user_index = user_distinct.tolist().index(userId)

for user in user_distinct[user_index:user_index+1]:

    user_tag_pref_all = user_tag_pref[user_tag_pref['user']==user]
    movie_distinct = np.unique(tf['movieId'])
    j = 1
    
    for movie in movie_distinct:
        
        if j%1000==0:
            print ("movie: ", j, "out of: ", len(movie_distinct), "with user: ", i, "out of: ", len(user_distinct))
        
        tf_movie = tf[tf['movieId']==movie]
        tag_merge = pd.merge(tf_movie, user_tag_pref_all, on = 'tag', how = 'left', sort = False)
        tag_merge['tag_pref'] = tag_merge['tag_pref'].fillna(0)
        tag_merge['tag_value'] = tag_merge['tag_vec']*tag_merge['tag_pref']
        
        tag_vec_val = np.sqrt(np.sum(np.square(tag_merge['tag_vec']), axis=0))
        tag_pref_val = np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
        
        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']]\
                                   .sum()\
                                   .rename(columns = {'tag_value': 'rating'})\
                                   .reset_index()
        
        tag_merge_final['rating']=tag_merge_final['rating']/(tag_vec_val*tag_pref_val)
        
        tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
        j=j+1
    
    i=i+1
    
tag_merge_all = tag_merge_all.sort_values(['user','rating'], ascending=False)


('movie: ', 1000, 'out of: ', 1572, 'with user: ', 1, 'out of: ', 609)


In [20]:
movies_rated = df_ratings[df_ratings['userId'] == userId]['movieId']
tag_merge_all = tag_merge_all[~tag_merge_all['movieId'].isin(movies_rated)]
tag_merge_all['user'] = tag_merge_all['user'].apply(np.int64)


In [21]:
tag_merge_all.head(10)

,user,movieId,rating
82,89,2295.0,0.292864
3,89,62.0,0.186961
67,89,1357.0,0.186961
77,89,2070.0,0.186961
131,89,6377.0,0.183781
22,89,594.0,0.180987
23,89,595.0,0.180987
24,89,596.0,0.180987
26,89,616.0,0.180987
46,89,1022.0,0.180987
